<a href="https://colab.research.google.com/github/JavanTang/Learn-a-little-tensorflow-every-day/blob/master/%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E2%80%94%E2%80%94Tensorflow%E5%AD%A6%E4%B9%A0%EF%BC%88%E5%9B%9B%EF%BC%89%E8%A7%A3%E5%86%B3%E5%9B%9E%E5%BD%92%E9%97%AE%E9%A2%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

首先我们先来学习什么是回归？

一样请移步 [回归与分类的区别](https://www.zhihu.com/question/21329754/answer/17901883)

看完之后，我首先来说说本章具体是做什么？

1. 首先来说说tensorflow官方的栗子做一个预测燃料效率的事情。
2. 然后我们去Kaggle上面做一个比赛House Prices: Advanced Regression Techniques


## 上代码

In [0]:
# Use seaborn for pairplot
!pip install seaborn
!pip install tensorflow==2.0.0-alpha0
import seaborn as sns
import pandas as pd
import tensorflow as t
from tensorflow import keras as k
from tensorflow.keras import layers
import matplotlib.pyplot as plt
print(t.__version__)

In [0]:
dataset_path = k.utils.get_file("auto-mpg.data", "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path


In [0]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin'] 
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset[:20]

上面的数据，我们看完了之后，需要去处理数据：
1. 将不完整的数据挑选出来（名词缺省值）
2. 将Origin地址，改成ONE-HOT

[ONE-HOT相关资料](https://blog.csdn.net/taotiezhengfeng/article/details/73692239)

In [0]:
# 统计缺省值
dataset.isna().sum()

In [0]:
# 处理这些缺省值，处理缺省值的方式有很多种，比如去掉，取中位数，平均值等等，我们选取的是去掉这些
dataset = dataset.dropna()


In [0]:
dataset.isna().sum()

In [0]:
# 将ORIGHT转换成ONE-HOT
# 这里是官方文档中处理的方式，其实已经有很多封装好了的方法，建议大家在这里可以采取自己的一些方式

origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

In [0]:
# Pandas分割数据集  如果不是Pandas的数据我们可以参考（三）中使用sklearn进行分割数据集
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [0]:
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]])


In [0]:
# describe 统计数值型数据的特性（数量、平均数、标准差、最小值，最大值）
train_stats = train_dataset.describe()
train_stats.pop("MPG")

# 这里之所以要转置是因为方便之后的矩阵相乘
train_stats = train_stats.transpose()
train_stats

In [0]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [0]:
train_stats

In [0]:
train_stats = train_stats.transpose()
def norm(x):
  '''
  标准化,参考3中有为什么要做标准化的解释
  '''
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [0]:
normed_train_data.head()

以上就是数据处理的部分了,下面我们要开始老套路 构建模型、配置模型、训练模型、验证模型。

In [0]:
# 构建模型
model = k.Sequential([
    layers.Dense(64, activation='relu',input_shape=[len(normed_train_data.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.summary()


# 配置模型
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_absolute_error', 'mean_squared_error'])



In [0]:
# 训练模型

history = model.fit(normed_train_data, train_labels,epochs=1000, validation_split = 0.2, verbose=0)

In [0]:
hist = pd.DataFrame(history.history)

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history)
  hist['epoch'] = list(hist.index+1)
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label='Val Error')
  plt.ylim([0,5])
  plt.legend()
  plt.show()
  
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [MPG^2]')
  plt.plot(hist['epoch'],hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'],hist['val_mean_squared_error'],
           label='Train Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()
  
  
plot_history(history.history)

在图中我们发现其实在100轮以后val（验证集）上面基本上没有明细的改进了，而训练集还在继续训练，这个时候我们为了确保它拥有很好的泛化性，我们应该在验证集不在变化的时候就停止训练。

这个工作在我们fit函数中就可以直接设置。

In [0]:
# 重新该块的工作，这里我们和上面的代码就重复了，今后我们会使用函数去build这个模型，以防遇到这种问题我们还需要重写这一块的代码
# 构建模型
model = k.Sequential([
    layers.Dense(64, activation='relu',input_shape=[len(normed_train_data.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.summary()


# 配置模型
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_absolute_error', 'mean_squared_error'])


# keras 函数回调的APIhttps://keras.io/zh/callbacks/
early_stop = k.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(normed_train_data, train_labels, epochs=1000, validation_split = 0.2, verbose=1, callbacks=[early_stop])


上面的因为verbose为1，所以输出了很多繁杂的数据。

我们想个办法让它30轮输出一次，我们可以创建一个回调函数
 [API](https://keras.io/zh/callbacks/)

In [0]:
class PrintLog(k.callbacks.Callback):
    def on_train_begin(self, logs):
        pass

    def on_epoch_end(self, epoch, logs):

        if epoch % 30 == 0: 
          print('epoch:',epoch, '\tlogs:',logs)


            
model = k.Sequential([
    layers.Dense(64, activation='relu',input_shape=[len(normed_train_data.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.summary()


# 配置模型
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_absolute_error', 'mean_squared_error'])


# keras 函数回调的APIhttps://keras.io/zh/callbacks/
early_stop = k.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(normed_train_data, train_labels, epochs=1000, validation_split = 0.2, verbose=0, callbacks=[PrintLog(),early_stop])
plot_history(history.history)

In [0]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))


In [0]:
test_predictions = model.predict(normed_test_data)
normed_test_data.head()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.ylim([0,50])
plt.xlim([0,50])

_ = plt.plot([-100, 100], [-100, 100])


看来真实值与预测值很接近

In [0]:
# 不加下面这句话会导致两者维度不一致
# https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.ndarray.flatten.html
test_predictions = test_predictions.flatten()
error = test_predictions - test_labels
plt.hist(error, bins = 25)

plt.xlabel('Prediction Error')
_ = plt.ylabel('Count')

这个图形不是那么的高斯，当然因为数据的问题导致的。

**总结一下：**

1. 回归问题可以使用均方差做loss。
2. 平均绝对误差是回归的平均标准。
3. 在上面我们将数字差别太大的值做了一遍缩放。
4. 数据不大的情况我们使用的隐藏层的层数尽量设置小一点，防止过拟合。
5. 早期终止可以避免过拟合。


练习：[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

这是一个房价预测的比赛,来源于Kaggle.

下次更新


# Reference
1. [Tensorflow官方教程](https://www.tensorflow.org/tutorials/keras/basic_regression#top_of_page)
2. [train_dataset.describe介绍](https://blog.csdn.net/j904538808/article/details/80747599)
3. [为什么要做标准化](https://www.zhihu.com/question/20467170)